### Notebook for training the SEGPVAE.


In [1]:
import torch
import os
import sys
import pickle

root = '/qfs/projects/atscale/atscale_dyn/'
sys.path.append(root + 'Code/atscale/GP/SEGP')
import TrainVAE_Tools as TVAE
import VAE as VAE
import SEGP as SEGP
import ModifyShape as ModShap
import ELBO as ELBO

import GP_Tools as GPT


In [2]:
# main

# user settings - data
N = 1 # dataset number to load
data_path = root + 'Data/GP/Data/Dataset{0}/'.format(N)
bs = 10
n_batch = 100
test_split = 0. # 0.2 # ratio of batches reserved for testing

# user settings - model
exp_no = 1 # experiment number
model_name = 'SEGPVAE/'
model_dir = root + 'Data/GP/Models/' + model_name + 'Exp_{:03d}/'.format(exp_no)

# B = None
C = None
# D = None
B = torch.tensor([[0, 0],
                  [0, 0],
                  [1, 0],
                  [0, 1]
                 ], dtype=torch.double)
        
# C = torch.tensor([[1, 0, 0, 0], 
#                   [0, 1, 0, 0]], dtype=torch.double)

D = torch.zeros((2,2), dtype=torch.double)

# user settings - training
max_epoch = 1
lr = 1e-2 # learning rate
wd = 1e-5 # weight decay
decay = 0.1 # scalar to multiply lr by at decay_epochs
decay_epochs = [75, 90] # epochs to perform lr cut

###################################################################################################

# Hardware agnostic settings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == 'cuda':
    print('Default tensor type is now cuda')
    torch.set_default_dtype(torch.float64)
    
print("Device in use is: ", device)


if os.path.isdir(model_dir):
    raise Exception("File already exists! Do not overwrite")
else:
    os.makedirs(model_dir)
    os.makedirs(model_dir + '/Encoder')
    os.makedirs(model_dir + '/GP')
    os.makedirs(model_dir + '/Decoder')


train_loader, test_loader, T, mean_U, mean_U_dt, exp_setup = TVAE.getData(data_path, test_split, bs, n_batch)

if model_name == 'SEGPVAE/':
    m = exp_setup['m'] # dimension of output
    n = exp_setup['n'] # dimension of state
    p = exp_setup['p'] # dimension of input
    lt = exp_setup['lt'] # length scale used for SE kernel of the input
    time = exp_setup['time'] # initial time, final time and integration step
    mean_x0 = torch.from_numpy( exp_setup['mean_x0'] ).to(dtype=torch.double)
    cov_x0 = torch.from_numpy( exp_setup['cov_x0'] ).to(dtype=torch.double)
    cov_eta = torch.from_numpy( exp_setup['cov_eta'] ).to(dtype=torch.double)
    tmax = len(T)
    py = exp_setup['py']
    px = exp_setup['px']
    h_dim = 500

    enc = VAE.VAEEncoder(py*px, h_dim, m)
    GP = SEGP.SEGP(m, n, p, lt, time, mean_x0, mean_U, mean_U_dt, cov_x0, cov_eta, tmax, B, C, D)
    dec = VAE.VAEDecoder(m, h_dim, py*px)

MS = ModShap.Modify_Shape(bs, m) # class for modifying shape of tensors.
criterion = ELBO.ELBO(bs, m) # We want to maximise this!
optimizer = torch.optim.Adam(list(enc.parameters()) + list(GP.parameters()) + list(dec.parameters()),
                             lr=lr, weight_decay=wd)

# log experiment settings
run_settings = {'N':N, 'bs':bs, 'n_batch':n_batch, 'test_split':test_split, \
               'exp_no':exp_no, 'model_name':model_name, 'model_dir':model_dir, \
                'max_epoch':max_epoch, 'lr':lr, 'wd':wd, 'decay':decay, \
                'decay_epochs':decay_epochs, 'device':device}

model_params = {'m':m, 'n':n, 'p':p, 'lt':lt, 'time':time, 'mean_x0':mean_x0, 'cov_x0':cov_x0, \
                'cov_eta':cov_eta, 'tmax':tmax, 'B':B, 'C':C, 'D':D, 'py':py, 'px':px, 'h_dim':h_dim}


with open(model_dir+'run_settings.pkl', 'wb') as f:
    pickle.dump(run_settings, f)
    f.close()
with open(model_dir+'model_params.pkl', 'wb') as f:
    pickle.dump(model_params, f)
    f.close()

model, stats = TVAE.train(train_loader, test_loader, MS, T, enc, GP, dec, optimizer, criterion, 
                          max_epoch, model_dir, decay, decay_epochs)


Default tensor type is now cuda
Device in use is:  cuda
mean_0 complete!
mean_1 complete!
K_00 complete!
K_01 complete!
K_11 complete!
A =  tensor([[-1., -0., -0., -0.],
        [-0., -1., -0., -0.],
        [-0., -0., -1., -0.],
        [-0., -0., -0., -1.]])
B =  tensor([[0., 0.],
        [0., 0.],
        [1., 0.],
        [0., 1.]])
C =  Parameter containing:
tensor([[ 1.3201, -0.3840, -0.1967, -0.0208],
        [ 1.3048, -0.4893,  0.2697, -1.9343]], requires_grad=True)
D =  tensor([[0., 0.],
        [0., 0.]])
Posterior computed!
Posterior sampled!
recon =  tensor(56872.5175, grad_fn=<DivBackward0>)
GCE =  tensor(1348.9943, grad_fn=<AddBackward0>)
lml =  tensor(-412.5122, grad_fn=<SubBackward0>)
loss =  tensor(-57808.9996, grad_fn=<NegBackward0>)
loss computed!
mean_0 complete!
mean_1 complete!
K_00 complete!
K_01 complete!
K_11 complete!
A =  tensor([[-1., -0., -0., -0.],
        [-0., -1., -0., -0.],
        [-0., -0., -1., -0.],
        [-0., -0., -0., -1.]])
B =  tensor([[0., 

RuntimeError: torch.linalg.eig: input tensor should not contain infs or NaNs.

In [ ]:
y_bounds = []
TVAE.plot_loss(model_dir, 'training', stats, max_epoch)